## MediPal -- Evaluation -- RAGAS

### In this section, I evaluate MediPal by RAGAS. I keep everything the same but change different set of models to see the differents. 

##### I set exact the same evaluation dataset, matrics and environment, comparing two sets of models. They have almost the same size but one set is general models, another one is fine-tuned on medical documents.

| Medical Team      |  General Team                      |
| ----------------- | ---------------------------------- |
| sentence-transformers/embeddinggemma-300m-medical | google/embeddinggemma-300m |
| ncbi/MedCPT-Cross-Encoder | cross-encoder/ms-marco-MiniLM-L6-v2 |
| ContactDoctor/Bio-Medical-Llama-3-8B | meta-llama/Meta-Llama-3-8B-Instruct |


##### Matrics: answer_relevancy, faithfulness, context_precision, context_recall, answer_correctness 

In [1]:
from src.medipal import medipal_app
from src.settings import AgentState
from datasets import Dataset
import pandas as pd
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision, context_recall, answer_correctness

Login HuggingFace!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


setup_retriever starts runing!
Login HuggingFace!
load_embedding_model starts runing!
load_embedding_model took 4.1428s
load_crossencoder starts runing!
load_crossencoder took 1.1978s
setup_retriever took 7.1813s


In [2]:
def print_metrics_table(metrics: dict):
    # Convert dict to DataFrame with two columns
    df = pd.DataFrame(metrics.items(), columns=["Metric", "Score"])
    # Round numbers to 4 decimal places
    df["Score"] = df["Score"].round(4)
    display(df)

# # Example input
# metrics = {
#     'answer_relevancy': 0.9281,
#     'faithfulness': 0.8235,
#     'context_precision': 0.8824,
#     'context_recall': 0.8235,
#     'answer_correctness': 0.7584
# }

# print_metrics_table(metrics)

In [3]:
qa_pairs = [
    {
        "question": "What conditions can phenylephrine help relieve?",
        "answer": "Phenylephrine can relieve nasal discomfort caused by colds, allergies, hay fever, and also help with sinus congestion and pressure."
    },
    {
        "question": "Does phenylephrine cure the cause of symptoms?",
        "answer": "No. Phenylephrine only relieves the symptoms but does not treat the underlying cause or speed up recovery."
    },
    {
        "question": "How does phenylephrine work in the body?",
        "answer": "Phenylephrine is a nasal decongestant. It works by reducing swelling of the blood vessels in the nasal passages."
    },
	{
        "question": "What allergies should you tell your doctor or pharmacist about before using pyrethrin and piperonyl butoxide shampoo?",
        "answer": "You should mention if you are allergic to pyrethrin, piperonyl butoxide, chrysanthemums, ragweed, any other medications, or any of the shampoo's ingredients."
    },
    {
        "question": "Why is it important to tell your doctor about other medications or supplements before using pyrethrin and piperonyl butoxide shampoo?",
        "answer": "Because prescription and nonprescription medications, vitamins, nutritional supplements, and herbal products may interact with the shampoo, so your doctor needs to know what you are taking or plan to take."
    },
    {
        "question": "What should you tell your doctor regarding your health history before using pyrethrin and piperonyl butoxide shampoo?",
        "answer": "You should tell your doctor if you have or have ever had skin conditions, or if you are pregnant, planning to become pregnant, or are breast-feeding."
    },
	{
        "question": "What should you do if someone swallows pyrethrin and piperonyl butoxide shampoo?",
        "answer": "Call your local poison control center immediately."
    },
    {
        "question": "What should you do if the person has collapsed or is not breathing after swallowing pyrethrin and piperonyl butoxide shampoo?",
        "answer": "Call local emergency services at 911 right away."
    },
    {
        "question": "Who should you contact first if pyrethrin and piperonyl butoxide shampoo is swallowed?",
        "answer": "You should first contact your local poison control center for instructions."
    },
	{
        "question": "What conditions can antipyrine and benzocaine otic help relieve?",
        "answer": "It can relieve ear pain and swelling caused by middle ear infections."
    },
    {
        "question": "Can antipyrine and benzocaine otic be used with antibiotics?",
        "answer": "Yes. It may be used along with antibiotics to treat an ear infection."
    },
    {
        "question": "How does the combination of antipyrine and benzocaine work?",
        "answer": "They are analgesics that work by reducing pain and discomfort in the ear."
    },
	{
        "question": "What is methylprednisolone injection used to treat?",
        "answer": "It is used to treat severe allergic reactions, multiple sclerosis, lupus, gastrointestinal disease, certain types of arthritis, and conditions affecting the blood, skin, eyes, nervous system, thyroid, kidneys, and lungs."
    },
    {
        "question": "Can methylprednisolone injection be used with other medications?",
        "answer": "Yes. It is sometimes used in combination with other medications to treat symptoms of low corticosteroid levels."
    },
    {
        "question": "What class of medications does methylprednisolone injection belong to?",
        "answer": "It belongs to a class of medications called corticosteroids."
    },
    {
        "question": "How does methylprednisolone injection help people with low corticosteroid levels?",
        "answer": "It replaces steroids that are normally produced naturally by the body."
    },
    {
        "question": "How does methylprednisolone injection treat other conditions?",
        "answer": "It works by reducing swelling and redness and by changing the way the immune system functions."
    },


    {
        "question": "Who should you contact first if pyrethrin and piperonyl butoxide shampoo is swallowed?",
        "answer": "You should first contact your local poison control center for instructions."
    },
    {
        "question": "What is ibuprofen used for?",
        "answer": "Ibuprofen is used to relieve pain, reduce inflammation, and lower fever."
    },
    {
        "question": "Can acetaminophen be taken on an empty stomach?",
        "answer": "Yes, acetaminophen can usually be taken on an empty stomach, but taking it with food may help avoid stomach upset."
    },
    {
        "question": "What is the main side effect of diphenhydramine?",
        "answer": "The main side effect of diphenhydramine is drowsiness."
    },
    {
        "question": "When should you take omeprazole?",
        "answer": "Omeprazole should be taken before meals, usually in the morning."
    },
    {
        "question": "What does insulin do in the body?",
        "answer": "Insulin helps glucose enter cells so it can be used for energy and lowers blood sugar levels."
    },
    {
        "question": "Can antibiotics treat viral infections like the flu?",
        "answer": "No, antibiotics only treat bacterial infections, not viral infections such as the flu."
    },
    {
        "question": "What should you avoid when taking metronidazole?",
        "answer": "You should avoid drinking alcohol while taking metronidazole and for at least 48 hours after finishing it."
    },
    {
        "question": "What is the purpose of a diuretic medication?",
        "answer": "Diuretics help your body eliminate excess salt and water through urine."
    },
    {
        "question": "What happens if you miss a dose of your birth control pill?",
        "answer": "Take the missed pill as soon as you remember, then continue your regular schedule."
    },
    {
        "question": "What is amoxicillin commonly prescribed for?",
        "answer": "Amoxicillin is commonly prescribed to treat ear, throat, lung, and urinary tract infections."
    },
    {
        "question": "Can you take aspirin with warfarin?",
        "answer": "You should not take aspirin with warfarin unless directed by your doctor, as it increases the risk of bleeding."
    },
    {
        "question": "What are common side effects of metformin?",
        "answer": "Common side effects of metformin include nausea, diarrhea, and stomach discomfort."
    },
    {
        "question": "Should antibiotics be stopped when you feel better?",
        "answer": "No, you should finish the full course even if you feel better to prevent resistance."
    },
    {
        "question": "What is loratadine used for?",
        "answer": "Loratadine is used to relieve allergy symptoms such as sneezing, runny nose, and itchy eyes."
    },
    {
        "question": "Can you drink grapefruit juice while taking statins?",
        "answer": "It’s best to avoid grapefruit juice while taking statins because it can increase the risk of side effects."
    },
    {
        "question": "What should you do if you develop a rash after taking penicillin?",
        "answer": "Stop taking the medication and contact your doctor immediately, as it could be an allergic reaction."
    },
    {
        "question": "How should insulin be stored?",
        "answer": "Unopened insulin should be kept in the refrigerator; opened vials can be kept at room temperature for about 28 days."
    },
    {
        "question": "What is prednisone used for?",
        "answer": "Prednisone is used to reduce inflammation in conditions such as asthma, arthritis, and allergic reactions."
    },
    {
        "question": "Can antibiotics cause diarrhea?",
        "answer": "Yes, antibiotics can sometimes cause diarrhea by upsetting the normal balance of bacteria in the intestines."
    },
    {
        "question": "What is the difference between brand-name and generic drugs?",
        "answer": "Generic drugs contain the same active ingredients as brand-name drugs and work the same way but usually cost less."
    },
    {
        "question": "Can I take acetaminophen with ibuprofen?",
        "answer": "Yes, they can be taken together for short-term pain relief, but follow dosing instructions carefully."
    },
    {
        "question": "What is the main use of an EpiPen?",
        "answer": "An EpiPen is used for emergency treatment of severe allergic reactions, known as anaphylaxis."
    },
    {
        "question": "Why should antibiotics be taken at regular intervals?",
        "answer": "Taking antibiotics at regular intervals helps maintain a constant level of the drug in your bloodstream for best results."
    },
    {
        "question": "What should you do if you vomit right after taking a medication?",
        "answer": "If you vomit within 15 minutes, you may need to repeat the dose; ask your pharmacist or doctor for advice."
    },
    {
        "question": "What is albuterol used for?",
        "answer": "Albuterol is used to relieve wheezing and shortness of breath in conditions like asthma and COPD."
    },
    {
        "question": "What are possible side effects of antihistamines?",
        "answer": "Common side effects include drowsiness, dry mouth, and dizziness."
    },
    {
        "question": "Can probiotics help after antibiotic treatment?",
        "answer": "Yes, probiotics may help restore healthy gut bacteria after antibiotic use."
    },
    {
        "question": "What is the recommended way to take iron supplements?",
        "answer": "Iron supplements are best taken on an empty stomach with water or orange juice for better absorption."
    },
    {
        "question": "What should you do if you think you overdosed on medication?",
        "answer": "Call emergency services or your local poison control center immediately for assistance."
    }	
]

##### Medical team first

In [4]:
def evaluation_rag(qa_pairs):
    questions = []
    model_answers = []
    retrieved_contexts_lists: list[list] = []
    reference_answers = []
    for i, s in enumerate(qa_pairs):
        question = s["question"]
        answer = s["answer"]
        questions.append(question)
        reference_answers.append(answer)    
        query = AgentState(query=question, session_id=1,wiki_used=False,brave_used=False,rewrite_counter=0,regenerate_counter=0)
        result = medipal_app.invoke(query) 
        retrieved_contexts_lists.append([result["retrieved_doc"]])
        model_answers.append(result["generation"])       
    data = Dataset.from_dict({
      "question": questions,                 # list[str]
      "answer": model_answers,               # list[str]
      "contexts": retrieved_contexts_lists,  # list[list[str]]
      "ground_truth": reference_answers      # list[str] (optional but needed for correctness)
    })
    # Everything is ready for evaluation
    result = evaluate(
      data,
      metrics=[answer_relevancy, faithfulness, context_precision, context_recall, answer_correctness],
      batch_size = 20
    )

    return result

In [ ]:
result = evaluation_rag(qa_pairs)

In [6]:
pd_result = result.to_pandas()

In [7]:
pd_result[:20]

,user_input,retrieved_contexts,response,reference,answer_relevancy,faithfulness,context_precision,context_recall,answer_correctness
0,What conditions can phenylephrine help relieve?,[phenylephrine may cause side effects. some si...,Phenylephrine is used to relieve nasal conges...,Phenylephrine can relieve nasal discomfort cau...,0.922988,0.0,0.0,0.0,0.417660
1,Does phenylephrine cure the cause of symptoms?,[phenylephrine is used to relieve nasal discom...,No,No. Phenylephrine only relieves the symptoms b...,0.729113,1.0,1.0,1.0,0.475050
2,How does phenylephrine work in the body?,[ask your pharmacist any questions you have ab...,Phenylephrine works by stimulating alpha-1 ad...,Phenylephrine is a nasal decongestant. It work...,0.909650,0.0,0.0,0.0,0.795048
3,What allergies should you tell your doctor or ...,[before using pyrethrin and piperonyl butoxide...,Before using pyrethrin and piperonyl butoxide...,You should mention if you are allergic to pyre...,0.942183,1.0,1.0,1.0,0.982308
4,Why is it important to tell your doctor about ...,[before using pyrethrin and piperonyl butoxide...,It is important to tell your doctor about oth...,Because prescription and nonprescription medic...,0.996401,0.5,1.0,1.0,0.675808
5,What should you tell your doctor regarding you...,[pyrethrin and piperonyl butoxide comes as a s...,Tell your doctor and pharmacist if you are al...,You should tell your doctor if you have or hav...,0.911794,1.0,1.0,1.0,0.198124
6,What should you do if someone swallows pyrethr...,[if someone swallows pyrethrin and piperonyl b...,if someone swallows pyrethrin and piperonyl b...,Call your local poison control center immediat...,1.000000,1.0,1.0,1.0,0.964629
7,What should you do if the person has collapsed...,[if someone swallows pyrethrin and piperonyl b...,911,Call local emergency services at 911 right away.,0.735170,0.0,1.0,1.0,0.957156
8,Who should you contact first if pyrethrin and ...,[if someone swallows pyrethrin and piperonyl b...,911,You should first contact your local poison con...,0.751902,1.0,1.0,1.0,0.191822
9,What conditions can antipyrine and benzocaine ...,[antipyrine and benzocaine otic comes as a sol...,Ear pain and swelling caused by middle ear in...,It can relieve ear pain and swelling caused by...,0.819735,1.0,1.0,1.0,0.820832


In [8]:
result

{'answer_relevancy': 0.8065, 'faithfulness': 0.6879, 'context_precision': 0.7447, 'context_recall': 0.7234, 'answer_correctness': 0.6059}

In [9]:
r = {'answer_relevancy': 0.8065, 'faithfulness': 0.6879, 'context_precision': 0.7447, 'context_recall': 0.7234, 'answer_correctness': 0.6059}
print("Medical Team's performance:")
print_metrics_table(r)

Medical Team's performance:


,Metric,Score
0,answer_relevancy,0.8065
1,faithfulness,0.6879
2,context_precision,0.7447
3,context_recall,0.7234
4,answer_correctness,0.6059


##### General Team's turn
##### I adjusted the models in src files.

In [ ]:
result = evaluation_rag(qa_pairs)

In [21]:
pd_result = result.to_pandas()

In [22]:
pd_result

,user_input,retrieved_contexts,response,reference,answer_relevancy,faithfulness,context_precision,context_recall,answer_correctness
0,What conditions can phenylephrine help relieve?,[phenylephrine may cause side effects. some si...,"I'm sorry, the provided documents do not conta...",Phenylephrine can relieve nasal discomfort cau...,0.000000,1.000000,0.0,0.000000,0.169839
1,Does phenylephrine cure the cause of symptoms?,[phenylephrine is used to relieve nasal discom...,"According to the retrieved documents, phenylep...",No. Phenylephrine only relieves the symptoms b...,0.870350,0.875000,1.0,0.500000,0.561988
2,How does phenylephrine work in the body?,[ask your pharmacist any questions you have ab...,"I'm sorry, the provided documents do not conta...",Phenylephrine is a nasal decongestant. It work...,0.000000,1.000000,0.0,0.000000,0.170515
3,What allergies should you tell your doctor or ...,[before using pyrethrin and piperonyl butoxide...,"According to the retrieved documents, you shou...",You should mention if you are allergic to pyre...,0.870142,1.000000,1.0,1.000000,0.985937
4,Why is it important to tell your doctor about ...,[before using pyrethrin and piperonyl butoxide...,"According to the retrieved documents, it is im...",Because prescription and nonprescription medic...,0.991313,1.000000,1.0,1.000000,0.972535
5,What should you tell your doctor regarding you...,[pyrethrin and piperonyl butoxide comes as a s...,"According to the retrieved documents, you shou...",You should tell your doctor if you have or hav...,0.977716,1.000000,1.0,1.000000,0.971152
6,What should you do if someone swallows pyrethr...,[if someone swallows pyrethrin and piperonyl b...,"According to the documents, if someone swallow...",Call your local poison control center immediat...,1.000000,1.000000,1.0,1.000000,0.722712
7,What should you do if the person has collapsed...,[if someone swallows pyrethrin and piperonyl b...,"According to the documents, if the person has ...",Call local emergency services at 911 right away.,0.990596,1.000000,1.0,1.000000,0.713622
8,Who should you contact first if pyrethrin and ...,[if someone swallows pyrethrin and piperonyl b...,"According to the documents, if someone swallow...",You should first contact your local poison con...,0.943457,1.000000,1.0,1.000000,0.597475
9,What conditions can antipyrine and benzocaine ...,[antipyrine and benzocaine otic comes as a sol...,"According to the retrieved documents, antipyri...",It can relieve ear pain and swelling caused by...,0.882064,1.000000,1.0,1.000000,0.827555


In [23]:
result

{'answer_relevancy': 0.6767, 'faithfulness': 0.9046, 'context_precision': 0.7447, 'context_recall': 0.6667, 'answer_correctness': 0.5508}

In [10]:
r = {'answer_relevancy': 0.6767, 'faithfulness': 0.9046, 'context_precision': 0.7447, 'context_recall': 0.6667, 'answer_correctness': 0.5508}
print("General Team's performance:")
print_metrics_table(r)

General Team's performance:


,Metric,Score
0,answer_relevancy,0.6767
1,faithfulness,0.9046
2,context_precision,0.7447
3,context_recall,0.6667
4,answer_correctness,0.5508
